In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier

# Read data

In [ ]:
cat_types={}

def process_data(filename):
    """
    first run sets cat_types
    """
    dd=pd.read_csv(filename).set_index("PassengerId")
    
    dd["Ticket2"]=dd["Ticket"].str[:2]
    dd["Cabin2"]=dd["Cabin"].str[:2]
    dd["Ticket_nums"]=dd["Ticket"].str.extract("([0-9]{3,})").astype(float)
    
    for cat_col in ["Sex", "Embarked", "Ticket2", "Cabin2"]:
        if cat_col not in cat_types:
            dd[cat_col]=dd[cat_col].astype("category")
            cat_types[cat_col]=dd[cat_col].cat.categories
        else:
            dd[cat_col]=dd[cat_col].astype("category").cat.set_categories(cat_types[cat_col])
    
    return dd

D=process_data("/kaggle/input/tabular-playground-series-apr-2021/train.csv")
D_test=process_data("/kaggle/input/tabular-playground-series-apr-2021/test.csv")
D

# Train

In [ ]:
feats=["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Fare", "Ticket2", "Cabin2", "Ticket_nums"]

X=D[feats]
y=D["Survived"]

model = LGBMClassifier(
    learning_rate=0.03,
    num_leaves=20,
    n_estimators=300,
)

model.fit(X, y);

# Predict

In [ ]:
X_test = D_test[feats]

pred = model.predict(X_test)

submission=pd.DataFrame({"PassengerId": X_test.index, "Survived": pred})
submission.to_csv("submission.csv", index=False)
submission.head()